# TP2 - Organización de Datos
#### Notebook principal

<hr>

### Notebooks utilizados:

- ***feature_engineering:*** notebook donde se generarán nuevos features para luego, realizar un proceso de selección de los mejores features para cada modelo.
- ***pre_processing:*** notebook para el manejo de los dataframes.
- ***parameter_tuning:*** notebook donde se tunean los parámetros. Por tratarse de un proceso costoso temporalmente, una vez obtenidos los mejores parámetros, los guardaremos en vez de generarlos cada vez que se quieran generar los resultados.
- ***predict:*** notebook que se encargará de generar el csv con las predicciones finales para el modelo que se le indique.

<hr>


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [2]:
import nbimporter

from pre_processing import load_train
import predict
import feature_engineering
import parameter_tuning

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from predict.ipynb
Importing Jupyter notebook from feature_engineering.ipynb
Importing Jupyter notebook from parameter_tuning.ipynb


<hr>

# Resultados obtenidos

### Modelo: Regresion lineal

In [3]:
# ...

### Modelo: Regresion logistica

In [3]:
# ...

### Modelo: SVM

In [4]:
# ...

### Modelo: Decision Tree

In [5]:
# ...

### Modelo: RandomForest

In [6]:
# ...

### Modelo: XGBoost

_Generacion del dataset de train con sus features_

In [3]:
import xgboost

In [4]:
df_train = feature_engineering.load_engineered_dataset()

In [5]:
Y = df_train.pop('precio').values
X = df_train.values

In [7]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.3)

In [11]:
reg = xgboost.XGBRegressor(max_depth=10,n_estimators=120 ,learning_rate=0.1, verbosity=2,subsample=0.9, min_child_weight=10)

In [12]:
reg.fit(X_train,Y_train)

[19:03:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:03:50] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1320 extra nodes, 0 pruned nodes, max_depth=10
[19:03:53] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1388 extra nodes, 0 pruned nodes, max_depth=10
[19:03:55] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1408 extra nodes, 0 pruned nodes, max_depth=10
[19:03:57] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1408 extra nodes, 0 pruned nodes, max_depth=10
[19:03:59] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1400 extra nodes, 0 pruned nodes, max_depth=10
[19:04:01] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1458 extra nodes, 0 pruned nodes, max_depth=10
[19:04:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roo

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=10, min_child_weight=10, missing=None, n_estimators=120,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=2)

_Comprobacion contra el conjunto de validacion_

In [13]:
Y_pred = reg.predict(X_val)
mean_absolute_error(Y_val,Y_pred)

595494.7659773763

### Modelo: CatBoost

In [3]:
# ...

### Modelo: LightGBM

In [3]:
# ...

### Modelo: KNN

In [3]:
# ...

### Modelo: Neural Networks

In [3]:
# ...

# testing

In [8]:
df = load_train()

In [11]:
df.head(3)

,titulo,descripcion,direccion,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,idzona,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Avenida Division del Norte 2005,Apartamento,Benito Juárez,Distrito Federal,NaN,2.0,1.0,2.0,80.0,80.0,23533.0,2015-08-23,0,0,0,0,0,2273000.0
1,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",AV. MEXICO,Casa en condominio,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,2.0,268.0,180.0,24514.0,2013-06-28,0,0,0,1,1,3600000.0
2,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Urbi Tonala,Casa,Tonalá,Jalisco,5.0,3.0,2.0,2.0,144.0,166.0,48551.0,2015-10-17,0,0,0,0,0,1200000.0
